In [20]:
from search import search_library
import random
from dotenv import load_dotenv
from openai import OpenAI, OpenAIError
import os

def ask(sys_msg, usr_msg):
    resp = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": usr_msg}
        ]
    )
    return resp.choices[0].message.content

def sample_text(text, x):
    max_start = max(0, len(text) - x)
    random_start = random.randint(0, max_start)
    excerpt = text[random_start:random_start + x]
    return excerpt


load_dotenv()
client = OpenAI()
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "o4-mini-2025-04-16")
DB_FILE = "library.sqlite"
MAX_RESULTS = 100

In [49]:
KEYWORDS = ["money", "economics"]
search_results = search_library(DB_FILE, KEYWORDS, MAX_RESULTS, match_all=True)
#everything = search_library(DB_FILE, None, max_results=5000)
money = {}
for e in search_results:
    title, text = e['title'], e['text']
    money[title] = text


#rem = ["Electrical Stimulation for Pelvic Floor Disorders", None, "Fundamental Neuroscience (3rd edition)", "Clinical Emergency Medicine", "Harrison's Principles of Internal Medicine 15th Edition"]
#for r in rem:
#    if r in hindgut:
#        del hindgut[r]

for title, text in money.items():
    print(title)
print(len(information))

Handbook on the Economics of Electricity
None
The Closing of the American Mind
Principles of Economics
Deep Learning
Reinventing the State: Economic Strategy and Institutional Change in Peru
Internet of Things and Big Data Technologies for Next Generation Healthcare
Elements of Information Theory: Second Edition
Energy and Civilization: A History
A Life of George Westinghouse
Fundamentals of Power System Economics
Signal Processing Techniques for Computational Health Informatics
The Dialectics of Dependency
Crassus: The First Tycoon
Chemistry: Atoms First 2e
The Invention of Enterprise: Entrepreneurship from Ancient Mesopotamia to Modern Times
Aging Issues in the United States and Japan
Davidson's Principles and Practice of Medicine, 24th Edition
An Introduction to Waste Management and Circular Economy
Green Banking and Green Central Banking
Marriage and Family
Encyclopedia of Artificial Intelligence: The Past, Present, and Future of AI
Crude Power: Politics and the Oil Market
The Evol

In [34]:
for _ in range(10):
    n = random.randint(0, len(information)-1)
    title, text = list(information.keys())[n], list(information.values())[n]
    excerpt = sample_text(text, 10000)
    summary = ask("Provide a one sentence summary of the following excerpt.", excerpt)
    try:
        dirt_hippo = ask("What hypothesis driven inquires can be performed based the following information?", excerpt)
    except:
        dirt_hippo = ""

    try:
        dirt_web = ask("What useful websites can be built based the following information?", excerpt)
    except:
        dirt_web = ""

    try:
        dirt_app = ask("What useful apps can be built based the following information?", excerpt)
    except:
        dirt_app = ""

    try:
        dirt_comm = ask("What successful commercial ventures can be built based the following information?", excerpt)
    except:
        dirt_comm = ""

    try:
        dirt_data = ask("Describe the most persuasive data‑visualization sequence to convey the excerpt’s insights.", excerpt)
    except:
        dirt_data = ""

    try:
        dirt_scifi = ask("Write a one‑sentence sci‑fi premise inspired by the excerpt.", excerpt)
    except:
        dirt_scifi = ""

    try:
        dirt_meta = ask("Generate three powerful metaphors that reframe the excerpt’s core idea.", excerpt)
    except:
        dirt_meta = ""

    try:
        dirt_ml = ask("How can machine learning be used in the following context?", excerpt)
    except:
        dirt_ml = ""
        
    try:
        dirt_saas = ask("What SAAS ventures can be built based the following information?", excerpt)
    except:
        dirt_saas = ""

    eid = add_entry(
        conn,
        title=title,
        excerpt=excerpt,
        summary=summary,
        texts=[
            dirt_hippo, dirt_web, dirt_app, dirt_comm, dirt_data, dirt_scifi, dirt_ml, dirt_saas
        ],
    )


In [33]:
print(title)
print(excerpt)

Engineered Multicellular Organisms
it should provide: the same
evolutionary algorithm can be reconfigured to design drugs
(15), autonomous machines (11, 13), metamaterials (16), or
architecture (17).

[0123] Here, we demonstrate a scalable approach for
designing living systems in silico using an evolutionary
algorithm, and we show how the evolved designs can be
rapidly manufactured using a cell-based construction toolkit.
The approach is organized as a linear pipeline that takes as
input a description of the biological building blocks to be
used and the desired behavior the manufactured system
should exhibit (FIG. 1). The pipeline continuously outputs
performant living systems that embody that behavior in
different ways. The resulting living systems are novel agegre-
gates of cells that yield novel functions: above the cellular
level, they bear little resemblance to existing organs or
organisms.

[0124] Results

[0125] The pipeline is organized as a sequence of genera-
tors and filters

In [27]:
import sqlite3, os, json

# Path for the SQLite database that will be created
db_path = '/Users/b/fantasiagenesis/crayon/boxes.sqlite'

# Create (or open) the database
conn = sqlite3.connect(db_path)
cur = conn.cursor()

# Create the main table for entry metadata
cur.execute('''
CREATE TABLE IF NOT EXISTS entries (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    excerpt TEXT,
    summary TEXT
)
''')

# Create a table for storing one‑to‑many "various texts" linked to each entry
cur.execute('''
CREATE TABLE IF NOT EXISTS texts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    entry_id INTEGER NOT NULL,
    body TEXT NOT NULL,
    FOREIGN KEY (entry_id) REFERENCES entries(id) ON DELETE CASCADE
)
''')

conn.commit()
#conn.close()


In [35]:
def add_entry(conn: sqlite3.Connection, title: str, excerpt:str, summary: str, texts: list[str]) -> int:
    """Insert an entry and a list of associated long texts. Returns the new entry ID."""
    cur = conn.cursor()
    cur.execute("INSERT INTO entries (title, excerpt, summary) VALUES (?, ?, ?)", (title, excerpt, summary))
    entry_id = cur.lastrowid
    cur.executemany(
        "INSERT INTO texts (entry_id, body) VALUES (?, ?)",
        [(entry_id, txt) for txt in texts],
    )
    conn.commit()
    return entry_id

def fetch_entry(conn: sqlite3.Connection, entry_id: int) -> dict:
    """Return a dict with entry metadata and a list of its texts."""
    cur = conn.cursor()
    cur.execute(
        """
        SELECT e.id, e.title, e.excerpt, e.summary, t.body
        FROM entries e
        LEFT JOIN texts t ON t.entry_id = e.id
        WHERE e.id = ?
        """,
        (entry_id,),
    )

    rows = cur.fetchall()
    if not rows:
        raise ValueError(f"No entry with id={entry_id}")

    # first row gives the metadata; collect all bodies
    meta = rows[0][:4]
    bodies = [row[4] for row in rows]
    entry = {"id": meta[0], "title": meta[1], "excerpt": meta[2], "summary": meta[3], "texts": bodies}
    return entry
    


In [43]:
from pprint import pprint

for n in range(1, 13):
    entry = fetch_entry(conn, n)
    pprint(entry['summary'])
    pprint(entry['texts'])

('They describe assembling and processing observational data on U.S. military '
 'and non‐military actions, carefully selecting confounding variables and '
 'using matching and fixed‐effects models under a selection‐on‐observables '
 'framework to approximate causal inference in the absence of randomization.')
['Below is a non‐exhaustive set of hypothesis‐driven inquiries you could '
 'pursue with your 234‐observation panel on U.S. coercive and non‐coercive '
 'actions.  I have grouped them by substantive focus and noted, in each case, '
 'the basic empirical approach you would take (all of these rest on your '
 'logit‐type model with selection‐on‐observables plus fixed effects for target '
 'state and/or dispute goal).\n'
 '\n'
 '1.  Effectiveness of Different Action Types  \n'
 '   Hypothesis 1.1 (“force vs. non‐force”):  After controlling for opponent '
 'capability, allies, domestic politics, etc., deployment of actual forces '
 '(border incursions, no-fly zones) yields a higher pr

In [56]:
m = 1
entry = fetch_entry(conn, m)
summary = entry['summary']
texts = entry['texts']
excerpt = entry['excerpt']
print(len(texts))
#pprint(entry['summary'])
#pprint(entry['texts'])

8


In [57]:
sys = f"""SYSTEM
You are an experienced data‑curation analyst.  
You turn free‑text research excerpts into tidy “codebooks” that list every variable needed for quantitative modelling.

USER
**Task**
1. Read the excerpt below (plus its 1‑sentence summary, if provided).  
2. Identify every concept that could vary across observations: outcomes, treatments, controls, IDs, time markers, diagnostics.  
3. For each concept create ONE row with four fields:
   • name          → snake_case, ≤ 25 chars, ready for code  
   • type          → binary | categorical | numeric | string | date  
   • coding/units  → short description of levels, transformations, or units  
   • description   → ≤ 15‑word human description (no line breaks)

4. If a concept is an obvious one‑hot dummy or a log‑transform, create that variable directly (e.g., `milcap_pc_log` or `goal_territorial`).

5. Omit anything that is clearly constant across all 234 observations.

6. Output the codebook as a **single Markdown table** with columns  
   | name | type | coding / units | description |

7. Beneath the table add a short “Notes” bullet list:
   – Any ambiguities or assumptions you made  
   – Any variables that may be redundant or collinear  
   – Any variable you suspect is needed but missing from the excerpt

**Excerpt begins**
{excerpt}
**Excerpt ends**

**Optional 1‑sentence summary (paste if you have one)**
{summary}
"""

data = ask(sys, "Answer the provided prompt.")
print(data)

| name                          | type        | coding / units                                                                 | description                                 |
|-------------------------------|-------------|--------------------------------------------------------------------------------|---------------------------------------------|
| observation_id                | numeric     | sequential unique identifier                                                   | unique row identifier                       |
| dispute_id                    | numeric     | original dispute code (1–115)                                                  | identifier for original dispute             |
| opponent_state                | string      | ISO country code or full name                                                  | target state of the US action               |
| campaign                      | string      | campaign name                                                                

In [58]:
sys = """SYSTEM:
You are a senior quantitative political‑science methodologist.  
USER:
Below is (A) a codebook and (B) a list of hypothesis statements.  
For **each** hypothesis, produce a reproducible analysis block in **exactly** the following Markdown template:

### Hypothesis <id>: <short label>
• Variables  
    – Treatment(s): …  
    – Outcome: …  
    – Key control(s): …  
• Model  
    – Equation: … (use plain-text math)  
    – Estimation details (link‑function, FE, clustering, etc.)  
• Diagnostic checks  
• Code skeleton (`statsmodels` Python)  
• Interpretation guide (≤40 words)

SPECIFICATIONS  
– Assume an N = 234 unbalanced panel; clustered SE on target‑state.  
– For every interaction, state expected sign.  
– If identification is doubtful, add a ⚠ “Identification caveat” line.  
– Keep each block under 150 words.

(A) DATA SCHEMA  
success (0/1) …  
{data}

(B) HYPOTHESES  
{hypotheses}
"""
sys = sys.format(data=data, hypotheses=texts[0])
#print(summary)
#print(texts[0])
steps = ask(sys, "Answer the given prompt.")
print(steps)

### Hypothesis 1.1: force_vs_nonforce  
• Variables  
  – Treatment(s): C(us_action) with “force_deployment” vs “verbal_threat”  
  – Outcome: success  
  – Key control(s): opponent_gdp_pc, opponent_mil_spend_pc, opponent_allies, us_ally_small, us_ally_great, presidential_approval, election_year, docs_statements_consistency, forces_pre_log  
• Model  
  – Equation: logit(success) = β₀+β₁·I(force_deployment)+β₂·I(verbal_threat)+…+α_opponent+γ_goal  
  – Estimation: logistic regression; opponent_fixed_effect & goal_fixed_effect; SE clustered by opponent_state  
• Diagnostic checks  
  – Pairwise contrast β₁>β₂; VIF for multicollinearity; Hosmer–Lemeshow  
• Code skeleton (`statsmodels` Python)  
```python
import statsmodels.formula.api as smf
model = smf.logit(
    "success ~ C(us_action) + opponent_gdp_pc + opponent_mil_spend_pc + "
    "opponent_allies + us_ally_small + us_ally_great + presidential_approval + "
    "election_year + docs_statements_consistency + forces_pre_log + "
    "

In [61]:
print(texts[0])
print(steps)

Below is a non‐exhaustive set of hypothesis‐driven inquiries you could pursue with your 234‐observation panel on U.S. coercive and non‐coercive actions.  I have grouped them by substantive focus and noted, in each case, the basic empirical approach you would take (all of these rest on your logit‐type model with selection‐on‐observables plus fixed effects for target state and/or dispute goal).

1.  Effectiveness of Different Action Types  
   Hypothesis 1.1 (“force vs. non‐force”):  After controlling for opponent capability, allies, domestic politics, etc., deployment of actual forces (border incursions, no-fly zones) yields a higher probability of success than purely verbal actions (direct diplomacy or threats).  
   – Empirical test:  Include dummies for each action type in a pooled logit; test pairwise contrasts (e.g., β_force_deploy > β_verbal_threat).

   Hypothesis 1.2 (“sanctions vs. diplomacy”):  Sanctions are more effective at achieving purely economic policy goals than direct 

In [70]:
x += 1
print(summary)
print(texts[x])

They describe assembling and processing observational data on U.S. military and non‐military actions, carefully selecting confounding variables and using matching and fixed‐effects models under a selection‐on‐observables framework to approximate causal inference in the absence of randomization.
On the brink of interstellar conflict, the United Earth Federation’s top data scientist must build a causal model that accounts for unobservable alien heterogeneities across dozens of star systems—only to discover a hidden confounding intelligence is manipulating their statistical assumptions to ignite a galactic war.


In [63]:
app = """1. Policy Outcome Predictor / Decision-Support Dashboard  
   • Target users: policymakers, embassy or theater planners, strategic analysts  
   • Core function: allow users to plug in a candidate “treatment” (e.g. verbal threat, sanctions, force deployment, no-fly zone) plus the key covariates (opponent GDP per capita, military capability, allied presence, domestic factors such as election year or approval rating, etc.) and display a probability of success  
   • Features:  
     – Dropdown menus for action type, policy goal, opponent state, rough force levels  
     – Auto-filled defaults or sliders for confounders (e.g. opponent military spending)  
     – Real-time output: estimated P(success) ± confidence interval, based on your logistic or matching model  
     – “What if?” toggles to compare two or more actions side-by-side  """


steps = ask("Give a step-by-step procedure to create the following app", app)
print(steps)

Here’s a high-level, step-by-step plan for building your Policy Outcome Predictor / Decision-Support Dashboard. Adjust tool choices and level of detail to fit your team’s expertise and timeline.

1. Requirements & Design  
   • Convene stakeholders (policymakers, analysts) to refine  
     – Key treatments (threat, sanctions, force, no-fly zone, etc.)  
     – Outcome definition (“success”) and acceptable error  
     – Covariates (GDP per capita, military spending, allied presence, domestic factors)  
     – UX needs: dropdowns, sliders, side-by-side comparison  
   • Sketch wireframes showing:  
     – Input panel (actions + covariate controls)  
     – Real-time output area (P(success) ± confidence interval)  
     – Comparison “What if?” section  

2. Data Collection & Preparation  
   • Identify data sources for historical cases:  
     – World Bank (GDP, demographics)  
     – SIPRI or Correlates of War (military capability, spending)  
     – Political timelines (election years,

In [66]:
comm = """1. Causal-Impact SaaS Platform for Policy & Strategy  
  • Product: A cloud-hosted dashboard that lets government affairs teams or defense ministries upload records of actions (e.g. threats, sanctions, deployments), define outcomes, and run instant causal-impact analyses.  
  • Key features:  
    – Automated confounder library (GDP, military spending, diplomatic posture, allies, etc.)  
    – Propensity-score matching, coarsened exact matching, inverse probability weighting  
    – Fixed-effects estimation by target, region or objective  
    – Scenario-comparison module (“What if we’d used diplomacy instead of an arms deployment?”)  
    – One-click export of charts, model diagnostics, policy briefs  
  • Revenue model: subscription tiers by number of users or “analyses” run per month, premium support for custom confounder inclusion.  """

steps = ask("Give a step-by-step procedure to create the following venture.", comm)
print(steps)

Below is a high-by-high, step-by-step blueprint for turning your “Causal-Impact SaaS Platform for Policy & Strategy” from concept into live service. Adjust timelines and team sizes to your budget and risk appetite.  

1. Validate the Opportunity  
   1.1. Conduct interviews with 10–15 target buyers (government affairs leads, defense analysts) to confirm pain points around “impact attribution,” report prep, and scenario analysis.  
   1.2. Survey existing tool usage and willingness-to-pay for on-demand causal analytics.  
   1.3. Refine your value proposition (e.g. “Cut uncertainty in strategy ROI estimates by 50%”).  

2. Competitive & Regulatory Scan  
   2.1. Map existing analytics vendors (DataRobot, CausalImpact R package, Palantir, Quid) and identify feature gaps.  
   2.2. Review data-privacy/IP laws (ITAR for defense data, GDPR if EU allies included, national security exemptions).  
   2.3. Document compliance requirements (auditable logs, data residency, user/role permissions).

In [69]:
vis = """1. Data Assembly Flowchart (Panel 1)  
   •  A simple left-to-right diagram with four boxes:  
     1) “Compile 115 disputes → 234 observations”  
     2) “Apply case-inclusion rules”  
     3) “Code 7 U.S. actions + 10 confounders”  
     4) “Fixed effects & matching”  
   •  Arrows between them, with a footnote “see pp. 196–197”  
   •  Purpose: orients the viewer to your four-step data‐prep process.  """

steps = ask("Describe the step-by-step process to create the following data visualization.", f"{summary}\n{vis}")
print(steps)

Here’s a recipe for building Panel 1 (“Data Assembly Flowchart”) from scratch. You can pick whatever tool you’re most comfortable with (PowerPoint/Keynote, Illustrator, Inkscape or code-based solutions like R’s DiagrammeR/mermaid or Python’s graphviz). I’ll outline a general, tool-agnostic workflow and then sketch out two concrete approaches (one GUI-based, one in R).

1. Clarify your content and layout  
   • Text for four boxes (left→right):  
     1) “Compile 115 disputes → 234 observations”  
     2) “Apply case-inclusion rules”  
     3) “Code 7 U.S. actions + 10 confounders”  
     4) “Fixed effects & matching”  
   • Arrows: simple single-headed connectors pointing from Box 1 → Box 2 → Box 3 → Box 4  
   • Footnote (centered below): “see pp. 196–197”  
   • Title (optional, above): “Panel 1. Data Assembly Flowchart”  

2. Choose your tool  
   – GUI option: PowerPoint, Keynote, Adobe Illustrator or free Inkscape  
   – Code option: R with DiagrammeR (mermaid syntax) or Python wi

In [ ]:
scifi = "On the brink of interstellar conflict, the United Earth Federation’s top data scientist must build a causal model that accounts for unobservable alien heterogeneities across dozens of star systems—only to discover a hidden confounding intelligence is manipulating their statistical assumptions to ignite a galactic war."
steps = ask("")

In [ ]:
# what are the existing markets, organizations, players? 
# who are the players? what is the game? the playbook? 